# Drifting Plasmas and the Numerical Cherenkov Instability

### Introduction

Imagine a typical laser-wakefield acceleration scenario: a laser pulse impinges on a plasma column much larger than the wavelength of the incident light. Now consider this same scenario in a frame relativistically drifting with Lorentz factor $\gamma$ in the direction of the laser pulse. The pulse will be redshifted, increasing its wavelength in the boosted frame; meanwhile, the length of the plasma column will be Lorentz-contracted by a factor of $\gamma$. Hence, the separation of the largest and smallest spatial scales, given by the ratio of the plasma length to the laser wavelength, is reduced. In addition, one can show that by the same token, the separation of the largest and smallest timescales, namely the light-crossing time of the plasma and the inverse laser frequency, are similarly reduced. (Note: if a moving window is used, then only the latter leads to computational speedup unless the box size can be considerably reduced in the boosted frame). 

When the math is carried out, this reduction of scale separations reduces the computational load by a factor of $\sim 2 \gamma^2 $ for small $\gamma$. Thus, running simulations in the boosted frame is an extremely attractive avenue for running otherwise prohibitively expensive simulations, even for small $\gamma$. 

A plasma initially stationary in the lab frame drifts relativistically in such a boosted frame. Let's run a simulation of such a relativistically drifting plasma on its own. The simulation is 2D and periodic in all directions. You can see the template deck that is being modified in `template-input-deck.os`. Basic parameters in this input deck are modified throughout this notebook.

As an aside: since these simulations need to run for a long time, I've made the default gride size very small (64x64), but still small enough to see the effect. This can be modified below via the `nx1` and `nx2` parameters. The 


### Drifting plasma with 2nd order Yee solver and no current filtering

In [1]:
import os 
import osiris 
import helpers
from IPython.display import Video, display 


def show_movie( sim_name ) : 
    path = sim_name + '/plots/nci-plots/nci-plots.mp4'
    v = Video( path, embed = True, width = 800 )
    display( v )


def run_osiris( sim_params, sim_name ) : 
    rundir = sim_name 
    inputfile = 'input.os'
    osiris.runosiris_2d(rundir=rundir, inputfile = inputfile )

    
    

In [2]:
# here are the parameters we will use in all the simulations (you can change them):
sim_params = helpers.NCIInputDeckOptions()

sim_params.solver_type = 'yee'
sim_params.use_current_filter = False

# sim params
sim_params.tmax = 200

# num frames per movie 
sim_params.num_output_data = 20 

# resolution parameters 
sim_params.nx1 = 64
sim_params.nx2 = 64
sim_params.dx1 = 0.8 
sim_params.dx2 = 0.8 
sim_params.dt = 0.2

# drift parameters
sim_params.gamma = 20 
sim_params.density = 1 

# yee solver params (2 = standard finite difference operator)
sim_params.yee_order = 2 

sim = 'yee-nofilter'

helpers.generate_input_deck( sim_params, sim )
run_osiris( sim_params, sim )
helpers.make_movie_individual( sim )
show_movie( sim )

runosiris completed normally
INFO: plotting indices: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]
INFO: corresponding timesteps: [  0  50 100 150 200 250 300 350 400 450 500 550 600 650 700 750 800 850
 900 950]
21
1
Making movie...
0 / 20
1 / 20
2 / 20
3 / 20
4 / 20
5 / 20
6 / 20
7 / 20
8 / 20
9 / 20
10 / 20
11 / 20
12 / 20
13 / 20
14 / 20
15 / 20
16 / 20
17 / 20
18 / 20
19 / 20
Saving movie...
ffmpeg -r 4.000000 -i yee-nofilter/plots/nci-plots//frames/%*.png -vcodec libx264 -crf 25 -pix_fmt yuv420p -y yee-nofilter/plots/nci-plots/nci-plots.mp4


### The above simulation demonstrates the NCI
You can see that there is a serious problem with the above simulation. The e2 and e1 electric field components grow exponentially. In position space, aphysical high-frequency disturbances can clearly be seen. In particular, in Fourier space (spatial Fourier transform at a fixed timestep), we can see that a distribution of modes are growing exponentially. 

In 2013, the above effect was derived and explained by Xinlu Xu, Peicheng Yu, and others in the paper "Numerical instability due to relativistic plasma drift in EM-PIC simulations", Computer Physics Communications, 2013. Starting with the numerical Vlasov Equation (i.e. the Vlasov equation with derivatives replaced by finite-difference operators) for the initial distribution function of a relativistically free-streaming plasma, they showed that simulation with the Yee solver admits unstable modes (i.e. Fourier modes with a nonzero imaginary part in time). The source of this growth is a coupling of the electron density fluctuation to the transverse electric field components, with a coupling strength that decays identically to 0 when the same calculation are done for the _exact_ Vlasov equation (i.e. with real derivatives rather than finite-difference operators). Thus, PIC simulations of drifting plasmas using the Yee field solver lead to aphysical growth of modes. The theory was confiremed as the authors demonstrated that the growth rate of these modes obtained numerically via root-finding methods matched closely those exhibited in the simulations. 



### First attempt at eliminating the NCI 

In the above 2013 paper, the instability was reduced by using an FFT-based solver in the longitudinal direction and applying a low-pass current filter; the combination of both of these led to satisfactory elimintation of the NCI. Unfortunately, by virtue of using an FFT along the longitudinal direction, this solver was not parallelizable in the longitudinal direction without heavy communications. 

The rest of this notebook is based on the approach to NCI mitigation presented by Fei Li, Peicheng Yu, Xinlu Xu, et al. in "Controlling the numerical Cerenkov instability in PIC simulations using a customized finite difference Maxwell solver and a local FFT based current correction", 2017, Computer Physics Communications. 

To start, let's run the same simulation with a higher-order Yee solver (16 points instead of 2) to see if that helps. 

In [3]:
sim_params.yee_order = 16 

sim = 'yee-highorder-nofilter'

helpers.generate_input_deck( sim_params, sim )
run_osiris( sim_params, sim )
helpers.make_movie_individual( sim )
show_movie( sim )

runosiris completed normally
INFO: plotting indices: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]
INFO: corresponding timesteps: [  0  50 100 150 200 250 300 350 400 450 500 550 600 650 700 750 800 850
 900 950]
21
1
Making movie...
0 / 20
1 / 20
2 / 20
3 / 20
4 / 20
5 / 20
6 / 20
7 / 20
8 / 20
9 / 20
10 / 20
11 / 20
12 / 20
13 / 20
14 / 20
15 / 20
16 / 20
17 / 20
18 / 20
19 / 20
Saving movie...
ffmpeg -r 4.000000 -i yee-highorder-nofilter/plots/nci-plots//frames/%*.png -vcodec libx264 -crf 25 -pix_fmt yuv420p -y yee-highorder-nofilter/plots/nci-plots/nci-plots.mp4


###  The high order solver does not solve the problem, but is the first step of the solution

You can see in the above that usage of the higher order solver causes the fastest-growing modes to only occur for large values of longitudinal wavenumber. This was not the case with the Yee solver, where the distribution of fastest-growing modes marched all the way into the $ k = (0,0) $ region. This is significant because in most problems of interest, the relevant physics is confined to the low-$k$ region, meaning that high wavenumber modes can be heavily filtered without altering physics. (Note: in general, the particular physics problem needs to be considered. For example, in high-harmonic generation, the relevant physics extends to high $k$, and hence filtering can't be used). 


### Next try: use of a longitudinal current filter for high-wavenumber modes
The below simulation is the same as the one above, except Fei's high-order spectral filter is added to filter current in the longitudinal direction. With the foresight that this is going to significantly reduce the above modes, we will also crank up the maximum simulation time far past the timescale for excitation of high-frequency modes in the previous simulations.

In [4]:
# increase max time with the foresight that this is going to (partially) work
sim_params.tmax = 3000

sim_params.use_current_filter = True 

# extend a range of 0.6 (i.e. k1 in [ -0.3, 0.3] k_g  )
sim_params.filter_limit = 0.6

# drop to ~0 in a width of 0.1 k_g 
sim_params.filter_width = 0.1 

# required parameter for FFT damping 
sim_params.n_damp_cell = 10

sim = 'yee-highorder-withfilter'

helpers.generate_input_deck( sim_params, sim )
run_osiris( sim_params, sim )
helpers.make_movie_individual( sim )
show_movie( sim )

runosiris completed normally
INFO: plotting indices: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]
INFO: corresponding timesteps: [    0   750  1500  2250  3000  3750  4500  5250  6000  6750  7500  8250
  9000  9750 10500 11250 12000 12750 13500 14250]
21
1
Making movie...
0 / 20
1 / 20
2 / 20
3 / 20
4 / 20
5 / 20
6 / 20
7 / 20
8 / 20
9 / 20
10 / 20
11 / 20
12 / 20
13 / 20
14 / 20
15 / 20
16 / 20
17 / 20
18 / 20
19 / 20
Saving movie...
ffmpeg -r 4.000000 -i yee-highorder-withfilter/plots/nci-plots//frames/%*.png -vcodec libx264 -crf 25 -pix_fmt yuv420p -y yee-highorder-withfilter/plots/nci-plots/nci-plots.mp4


### The spectral filter does its job, but the next NCI mode needs to be addressed

As you can see in the above simulation, the spectral filter does its job. The high frequency modes have been eliminated from the simulation all all times, and in particular at comparable timescales the spectral magnitudes are considerably lower. 

However, as we run the simulation for a larger time, a new problem becomes apparent. This is the excitation of a new mode around $k~(0.2, 0) $, which evidentally grows slower than the high-frequency modes we saw before as it takes longer to start growing. After $t \sim 1000$, exponential growth of this mode is apparent, and at $t~ 3000$ the fully-developed instability can clearly be seen as strong oscillations in the E2 position space. Plotting NCI modes on their own reveals that the high-frequency (fastest-growing) mode is the $(0,0)$ mode and the second fastest which we see here is the $(0,-1)$ mode. 

In this case, filtering cannot be used because the mode is too close to those where physics resides. So, any attempt to address this instability must preserve physics near the $(0,-1)$ mode (to reiterate, filtering does not preserve physics where it is applied). 

### Next attempt: the bump solver 
The key insight of the 2017 paper mentioned above is that this problem can be solved by slightly perturbing the above 16th order Yee solver so that the numerical speed of light is slightly faster than the speed of light (this is the "bump") in the region of the $(0,-1)$ mode. This causes the NCI dispersion relation to have real solutions (i.e. stable modes) in the vicinity of the bump. The below simulation uses this technique along with the current filter we used previously.  

In [5]:
# bump parameters 
sim_params.solver_type = 'bump'
sim_params.bump_order = 16
sim_params.bump_n_coef = 16
sim_params.klower = 0.1 
sim_params.kupper = 0.3 
sim_params.dk = 0.005

sim = 'bump-withfilter'

helpers.generate_input_deck( sim_params, sim )
run_osiris( sim_params, sim )
helpers.make_movie_individual( sim )
show_movie( sim )

runosiris completed normally
INFO: plotting indices: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]
INFO: corresponding timesteps: [    0   750  1500  2250  3000  3750  4500  5250  6000  6750  7500  8250
  9000  9750 10500 11250 12000 12750 13500 14250]
21
1
Making movie...
0 / 20
1 / 20
2 / 20
3 / 20
4 / 20
5 / 20
6 / 20
7 / 20
8 / 20
9 / 20
10 / 20
11 / 20
12 / 20
13 / 20
14 / 20
15 / 20
16 / 20
17 / 20
18 / 20
19 / 20
Saving movie...
ffmpeg -r 4.000000 -i bump-withfilter/plots/nci-plots//frames/%*.png -vcodec libx264 -crf 25 -pix_fmt yuv420p -y bump-withfilter/plots/nci-plots/nci-plots.mp4


### Success

You can see that in the above simulation, the high-frequency modes are still eliminated, while use of the bump solver has eliminated the $(0,-1)$ NCI mode. Electric field energy growth has been reduced to an acceptable level.

In practice, rather than using this somewhat time-consuming procedure, a script can be used to generate the NCI dispersion relation for given simulation parameters, i.e. visualize in advance where the (0,0) and (0,-1) modes will be in advance without running a simulation. The appropriate bump solver parameters can be selected by rerunning the NCI dispersion relation script using the bump solver coeficients until suitable ones are found. For example, too large a bump can significantly modify the physics in regions of interest -- the only way to see is to experiment with the parameters by hand). Then merely one test simulation with the chosen NCI mitigation parameters is required to verify that the NCI is suppressed for timescales of interest. 